In [12]:
import torch
from dist_ae import SetAutoencoderGNN, train_w_stop
from distns import distns

distn_name = 'poisson'
generate_params = distns[distn_name]['generate_params']
sample_distn = distns[distn_name]['sample']
fr_dist = distns[distn_name]['fr_dist']

In [13]:
n_sets = 1000
set_size = 10**3
n_features = 2

params_train = generate_params(n_sets, n_features)
params_val = generate_params(n_sets, n_features)
params_test = generate_params(n_sets, n_features)

x_train = sample_distn(*params_train, n_sets, set_size, n_features)
x_val = sample_distn(*params_val, n_sets, set_size, n_features)
x_test = sample_distn(*params_test, n_sets, set_size, n_features)

In [14]:
dist_ae = SetAutoencoderGNN(n_features, 16, 64, set_size)
optimizer = torch.optim.Adam(dist_ae.parameters(), lr=5e-4)

# create dataloader
train_loader = torch.utils.data.DataLoader(torch.from_numpy(x_train).float(), batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(torch.from_numpy(x_train).float(), batch_size=32, shuffle=False)

dist_ae = train_w_stop(dist_ae, optimizer, 
                       train_loader, val_loader,
                       max_epochs = 300, device='cuda')

Epoch 1 | Train Loss: 6468.4585 | Val Loss: 5164.1876
Epoch 2 | Train Loss: 4140.1424 | Val Loss: 3926.2941
Epoch 3 | Train Loss: 3899.8107 | Val Loss: 3819.6408
Epoch 4 | Train Loss: 3830.5583 | Val Loss: 3793.8885
Epoch 5 | Train Loss: 3803.6861 | Val Loss: 3754.7501
Epoch 6 | Train Loss: 3757.5527 | Val Loss: 3688.7012
Epoch 7 | Train Loss: 3430.1451 | Val Loss: 2305.0439
Epoch 8 | Train Loss: 1735.7031 | Val Loss: 1460.0293
Epoch 9 | Train Loss: 1429.1621 | Val Loss: 1368.1817
Epoch 10 | Train Loss: 1370.8586 | Val Loss: 1314.5335
Epoch 11 | Train Loss: 1345.7376 | Val Loss: 1310.2297
Epoch 12 | Train Loss: 1312.4787 | Val Loss: 1341.8053
Epoch 13 | Train Loss: 1308.8187 | Val Loss: 1302.0059
Epoch 14 | Train Loss: 1311.5075 | Val Loss: 1198.2176
Epoch 15 | Train Loss: 1037.0055 | Val Loss: 893.6628
Epoch 16 | Train Loss: 869.5776 | Val Loss: 882.3692
Epoch 17 | Train Loss: 839.7954 | Val Loss: 841.2095
Epoch 18 | Train Loss: 822.9946 | Val Loss: 789.2568
Epoch 19 | Train Loss: 797

In [15]:
z, rec = dist_ae(torch.from_numpy(x_test).float().cuda())

In [16]:
rec.mean(axis=1)

tensor([[23.6147, 17.6346],
        [11.2376, 31.7255],
        [94.2331, 24.5842],
        ...,
        [37.1085, 22.8546],
        [68.7688, 75.1813],
        [63.3654, 59.6592]], device='cuda:0', grad_fn=<MeanBackward1>)

In [17]:
params_test[0]

array([[22.54693035, 18.49381276],
       [ 9.95508808, 28.2864475 ],
       [88.87211816, 23.27843698],
       ...,
       [35.66889918, 23.69641599],
       [68.26928727, 69.69083164],
       [65.40117993, 61.03956331]])